<a id = '구성전략'></a>
## 구성 전략 및 위기관리 전략

### 구성전략

- <strong>수익률 확인</strong>
    - 이전 스텝의 포트폴리오를 확인한다.
    - 이전 스텝의 포트폴리오의 현재 수익률과 수익총액을 확인한다.
    - 이전 스텝의 총 자산금액을 5(top5)로 나누거나, 가중치에 따라 포트폴리오의 자산별 구매금액을 확인한다.
        - 이때 자산별 구매금액을 `종목비중금액`(thres)이라고 하자.

- <strong>포트폴리오 리밸런싱</strong>
    - 다음 스텝의 포트폴리오를 확인한다.
    - sell
        - 유지 종목중 종목비중금액 기준보다 높은 가격을 가지고 잇는 주식은 여유분을 판매
        - 유지되지 않는 종목은 전 종목을 모두 판매
        - 판매한 금액은 예치금(bank)에 저장해둔다.
    - buy
        - 유지되지만 종목비중금액 기준을 넘지 않는 종목은 예치금을 통해 부족분을 구매한다.
        - 새롭게 추가되는 종목은 예치금에서 종목비중금액 기준이 부족하지 않은만큼 구매한다. 
        - 예치금이 부족하다면, 상위 모멘텀 종목부터 구매할 수 있도록 한다.
    - 수수료
        - 판매(sell)이나 구매(buy)를 할때마다 각 금액별 수수료를 계산해 차감하도록 한다.
        - 수수료는 0.015%를 기준으로 판매,구매를 수행한다.  
    - 위기관리
        - 위험구간으로 판단되는 구간에서는 보유자산의 50%를 예치금에 넣어두고 위기구간이 끝나기 전까지 사용하지 않는다.
        - 나머지 50%금액으로는 이전과 동일하게 리밸런싱을 수행한다.
        

In [1]:
import pandas as pd

In [2]:
def profit_calcul(profit_table,change_price,next_day):
    profit_list=[]
    for i in profit_table.stock:
        
        #  종목이 없을 경우에는 pass.
        if pd.isnull(i):
            profit_list.append([np.nan,0])
        
        #  가격 변동량 기반으로 시장가치를 측정.
        else:
            market_value = profit_table[profit_table.stock==i].profit.values[0]
            single_today_profit = market_value * change_price[change_price.date==next_day][i].values[0]
            profit_list.append([i,single_today_profit])
    profit_table = pd.DataFrame(profit_list,columns=["stock","profit"])
            
    return profit_table

In [3]:
# 판매 전용 function
def sell(profit_table,next_list,bank,fee=0.00015,anomaly = False,weight=pd.DataFrame([])):

    # 현재 보유종목, 이후 유지종목
    today_list = list(profit_table.stock)
    keep_list = list(set.intersection(set(today_list), set(next_list)))
    
    # 아직 투자가 진행되지 않은 상황에서는 기본값을 리턴한다.
    if pd.isnull(today_list[0]):
        return (profit_table.reset_index(drop=True), bank, None)
    
    # 이상구간으로 판정되었을때 수익구간의 수익을 분배해준다.
    if anomaly == True:
        total_profit = np.sum(profit_table.profit) * 0.5
    else:
        total_profit = np.sum(profit_table.profit)
        
    # 새롭게 각 종목별 동일 비중으로 분배할 금액
    new_divide = total_profit/len(next_list)
    
    # 종목별 처리(판매)
    for i in today_list:
        
        # weight에 의한 divide 계산
        if len(weight)!=0:
            new_divide = total_profit * weight[i]
        else:
            new_divide = total_profit/len(next_list)
            
        # 초기 단계일 경우 단계를 건너뛴다.
        if pd.isnull(i): continue
            
        # 종목 시장가치
        market_value = profit_table[profit_table.stock==i].profit.values[0]

        # 유지되는 종목 밸런스 확인
        if i in keep_list:
            # 유지종목의 가격이 높은 가격을 가지고 있는 경우 여유분 판매
            if market_value > new_divide:
                profit = market_value - new_divide
                margin = profit * (1-fee)
                bank += margin
                # profit 재계산
                profit_table.loc[profit_table['stock'] == i, 'profit'] = new_divide
                
            # 유지는 하지만 그닥 높은 가격이 아닐경우에는 추가판매 없음
            else:
                pass

        # 유지되지 않는 종목 일괄 판매
        else:
            margin = market_value * (1-fee)
            bank += margin
            # 일괄 판매 종목 리스트제거
            profit_table = profit_table[profit_table.stock != i]
    
    return(profit_table.reset_index(drop=True), bank, new_divide)

In [4]:
def buy(profit_table, next_list, bank, new_divide=None,fee=0.00015,weight=pd.DataFrame([])):
    
    # 현재 보유종목, 이후 유지종목
    today_list = list(profit_table.stock)
    keep_list = list(set.intersection(set(today_list),set(next_list)))
    
    # null check point
    if len(profit_table)==0: 
        null_checkpoint='all_sell'
    else: 
        null_checkpoint = profit_table.stock[0]
    
    # 포트폴리오 투자 활성화 전에는 function을 종료한다. (현재스텝 x, 다음스텝 포트폴리오 x)
    if pd.isnull(null_checkpoint) and (len(next_list)==0):
        return profit_table, bank
    # 첫투자 활성화를 위해서 포트폴리오를 초기화한다. (현재스텝 x, 다음스텝 포트폴리오 o)
    elif pd.isnull(null_checkpoint) and  (len(next_list)!=0):
        profit_table = profit_table[0:0]
        new_divide = bank / len(next_list)
    # 그 외에는 정상적으로 리밸런싱 구매를 진행.
    else:
        new_divide = (new_divide*len(next_list) + bank)/len(next_list)
             
    # 구매 프로세스 시작 
    for i in next_list:
        
        # weight에 의한 new_divide 계산
        if len(weight)!=0:
            new_divide =  (np.sum(profit_table.profit)+ bank/(1-fee) ) * weight[i]
        else:
            pass
        
        # 유지되는 종목
        if (i in keep_list):
            
            # 현재 시장가치 
            market_value = profit_table[profit_table.stock==i].profit.values[0]
            
            # 유지종목의 가격이 높은 가격을 가지고 있는 경우 리밸런싱
            if market_value < new_divide:
                
                # 구매해야하는돈이 뱅크에 충분한 경우
                if (new_divide - market_value) < bank:
                    order = new_divide - market_value
                    buy = order * (1-fee)
                    bank -= order
                    profit_table.loc[profit_table['stock'] == i, 'profit'] = market_value + buy
                    
                # 구매해야하는 돈이 뱅크만큼 없는 경우
                else:
                    order = bank
                    buy = order * (1-fee)
                    bank = 0 
                    profit_table.loc[profit_table['stock']==i, 'profit'] = market_value + buy
                    
            # 유지종목의 가격이 충분한 경우 (이미 sell 단계에서 처리됨)
            else:
                pass

        # 새롭게 추가되는 종목은 일괄 구매
        else:
            order = min(bank, new_divide)
            buy = order * (1-fee)
            bank -= order
            profit_table = profit_table.append({'stock':i, 'profit':buy},ignore_index = True)
            
    return(profit_table.reset_index(drop=True), bank)